In [1]:
import pandas as pd;
import numpy as  np;
from scipy.optimize import minimize;
import random;

In [2]:
ratings_small = np.genfromtxt('ml-latest-small//ratings.csv',delimiter=',',skip_header =1)[:,0:3]

In [3]:
def get_rhat(u,v):
    return np.matmul(u,np.transpose(v))

In [4]:
def cal_error(r,rhat):
    return 0.5 * np.sum(np.square(r - rhat))

In [5]:
def matrix_factorization(ratings,alpha,steps,concept=3):
    r=ratings
    row = (r.shape[0])
    col = (r.shape[1])
    
    np.random.seed(1234)
    u = np.random.rand(row,concept)
    v = np.random.rand(col,concept)
    
    e = r - get_rhat(u,v)
    e = np.array(pd.DataFrame(e).fillna(0))
    #e1 = np.zeros((row,col))
    #
    #for i in range(row):
    #    for j in range(col):
    #        if r[i][j] > 0:
    #            e1[i,j] = r[i][j] - get_rhat(u,v)[i,j]
    #           
               
    #e = r - get_rhat(u,v)
    #print(e)
    #print(e1)
    for s in range(steps):
        u_c = u + alpha * np.matmul(e,v)
        v_c = v + alpha * np.matmul(np.transpose(e),u)
       
        u = np.copy(u_c)
        v = np.copy(v_c)
        e = r - get_rhat(u,v)
        e = np.array(pd.DataFrame(e).fillna(0))
    return get_rhat(u,v)

In [6]:
R = [[5,3,np.nan,1],
  [4,np.nan,np.nan,1],
  [1,1,np.nan,5],
  [1,np.nan,np.nan,4],
  [np.nan,1,5,4]]
R = np.array(R)

In [7]:
matrix_factorization(R,0.002,1000)

array([[5.00479217, 2.98757696, 3.29128869, 0.99876123],
       [3.99561129, 2.12559217, 3.00200174, 0.99823669],
       [1.01425366, 0.96503099, 4.64540656, 4.997607  ],
       [0.99271356, 0.4237421 , 4.14724211, 3.99633337],
       [2.37957614, 1.04454135, 4.98434807, 4.01184795]])

In [8]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [72]:
r2 = ratings.pivot(index='userId', columns='movieId', values='rating');
r3 = np.asarray(r2)

In [10]:
matrix_factorization(r3,0.0002,1000)

array([[4.77956271, 4.23373704, 3.86487266, ..., 3.73295744, 3.85884154,
        4.75444991],
       [3.86327595, 3.45850166, 3.22185477, ..., 2.96080616, 3.10413529,
        3.61222324],
       [2.53267637, 2.31260267, 2.01775225, ..., 1.52244183, 1.13257878,
        1.45784672],
       ...,
       [3.7007829 , 3.29346918, 2.95997503, ..., 2.74781452, 2.68008488,
        3.37167088],
       [3.48792375, 3.13277312, 2.95236763, ..., 2.68270533, 2.86314554,
        3.2417244 ],
       [4.15047787, 3.7149173 , 3.45576772, ..., 3.17597563, 3.31991141,
        3.87439329]])

In [24]:
indice = list()

for i in range(r3.shape[0]):
    for j in range(r3.shape[1]):
        if np.isnan(r3[i][j]) == False:
            indice.append([i,j])

In [26]:
len(indice)

100836

In [28]:
r3.shape[0]*r3.shape[1]

5931640

In [30]:
(100836/5931640)*100

1.6999683055613624

In [ ]:
ratio = 0.1

In [41]:
random.seed(2018)
test = random.sample(indice, int(ratio*len(indice)))

In [46]:
#train = [x for x in indice if x not in test] 

In [43]:
len(train)

90753

In [44]:
len(test)

10083

In [45]:
len(train)+len(test)

100836

In [38]:
len(indice)

100836

In [39]:
indice[0:10]

[[0, 0],
 [0, 2],
 [0, 5],
 [0, 43],
 [0, 46],
 [0, 62],
 [0, 89],
 [0, 97],
 [0, 124],
 [0, 130]]

In [85]:
def mutate_ratings(ratings,test):
    rt = ratings.copy()
    for i in test:
        rt[i[0]][i[1]] = np.nan
    return (rt)

In [86]:
new_r3 = mutate_ratings(r3, test)

In [92]:
import time

start = time.time()

pred_new_r3_2 = matrix_factorization(new_r3,0.0002,1000, concept = 2)
pred_new_r3_3 = matrix_factorization(new_r3,0.0002,1000, concept = 3)
pred_new_r3_4 = matrix_factorization(new_r3,0.0002,1000, concept = 4)
pred_new_r3_5 = matrix_factorization(new_r3,0.0002,1000, concept = 5)

end = time.time()
print(end-start)

-0.00485684908926487


In [99]:
def fetch_ratings(ratings, predicted, test):
    y_true = list()
    y_pred = list()
    for i in test:
        y_true.append(r3[i[0]][i[1]])
        y_pred.append(predicted[i[0]][i[1]])
    return(y_true,y_pred)

In [100]:
y_true,y_pred_2 = fetch_ratings(r3, pred_new_r3_2, test)
y_true,y_pred_3 = fetch_ratings(r3, pred_new_r3_3, test)
y_true,y_pred_4 = fetch_ratings(r3, pred_new_r3_4, test)
y_true,y_pred_5 = fetch_ratings(r3, pred_new_r3_5, test)

In [101]:
y_pred_2[1:10]

[3.206484027879532,
 1.3971523443972549,
 3.726042259529151,
 4.404169339307783,
 2.776061332390903,
 3.3537139174352304,
 3.3231249789847137,
 3.775852238192824,
 3.3484724613045733]

In [90]:
from sklearn.metrics import mean_squared_error

In [102]:
mse_2 = mean_squared_error(y_true, y_pred_2)
mse_3 = mean_squared_error(y_true, y_pred_3)
mse_4 = mean_squared_error(y_true, y_pred_4)
mse_5 = mean_squared_error(y_true, y_pred_5)

In [103]:
print(mse_2, mse_3, mse_4, mse_5)

0.848772461879257 0.8397398922443507 0.8238504890637401 0.8343180262188046


In [79]:
y_true[1:10]

[4.0, 3.0, 4.0, 4.0, 5.0, 2.5, 5.0, 2.5, 3.0]

In [80]:
y_pred[1:10]

[3.1858993109954072,
 1.974794244302193,
 3.7444569243284436,
 4.386698417341947,
 3.1248248987591034,
 3.092258016869915,
 3.3643979574013887,
 3.8048116961426373,
 3.3077430188381802]

In [68]:
#pred_new_r3

array([[4.79716714, 4.1708642 , 4.01390197, ..., 3.68402278, 3.83286809,
        4.71426636],
       [3.89872983, 3.47294289, 3.26508031, ..., 2.93195664, 3.08545198,
        3.63996295],
       [2.46626382, 2.45709606, 1.4218214 , ..., 1.6219774 , 1.34280756,
        1.67594919],
       ...,
       [3.63425273, 3.27746473, 2.67282671, ..., 2.68113983, 2.56738672,
        3.28467297],
       [3.51024972, 3.13206744, 3.01228431, ..., 2.64021905, 2.83343069,
        3.26849018],
       [4.20696803, 3.73513869, 3.54587439, ..., 3.17436275, 3.35206065,
        3.95721243]])